In [21]:
import pandas as pd
import numpy as np

In [22]:
# load and shift data so score corrisponds to next day
data_combined = pd.read_csv('./data/Combined_News_DJIA.csv')
data_combined.Label = data_combined.Label.shift(1)
data_combined = data_combined[1:]
stories = [c for c in data_combined if c.startswith('Top')]
data_combined = pd.melt(data_combined, id_vars=['Date','Label'], value_vars=stories, value_name='Headline')
#data_combined = data_combined.drop(['variable'], axis=1)
data_combined['Headline'] = data_combined['Headline'].apply(lambda x: str(x))
data_combined.head()

,Date,Label,variable,Headline
0,2008-08-11,0.0,Top1,b'Why wont America and Nato help us? If they w...
1,2008-08-12,1.0,Top1,b'Remember that adorable 9-year-old who sang a...
2,2008-08-13,0.0,Top1,b' U.S. refuses Israel weapons to attack Iran:...
3,2008-08-14,0.0,Top1,b'All the experts admit that we should legalis...
4,2008-08-15,1.0,Top1,"b""Mom of missing gay man: Too bad he's not a 2..."


In [23]:
# randomize
random_data = data_combined.sample(frac=1)

# split data
TT_SPLIT = .8
index = int(TT_SPLIT*len(data_combined))
train = random_data[:index]
test = random_data[index:]

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=100, norm='l2', encoding='latin-1', 
                        ngram_range=(1, 4), stop_words='english')

train_vectors = tfidf.fit_transform(train.Headline).toarray()
labels = data_combined.Label

test_vectors = tfidf.transform(test.Headline)

In [26]:
from keras.preprocessing.text import Tokenizer

# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=200)
tokenizer.fit_on_texts(train.Headline)
sequences_train = tokenizer.texts_to_sequences(train.Headline)
sequences_test = tokenizer.texts_to_sequences(test.Headline)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/preprocessing/text.py:172: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [27]:
from keras.preprocessing import sequence
from keras.utils import np_utils


maxlen = 50

y_train = np.array(train.Label)
y_test = np.array(test.Label)


print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)

Y_train = np_utils.to_categorical(y_train, 2)
Y_test = np_utils.to_categorical(y_test, 2)


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
X_train shape: (39760, 50)
X_test shape: (9940, 50)


In [ ]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import ConvLSTM2D
from keras.layers.core import Dense, Activation


print('Build model...')
model = Sequential()
model.add(Embedding(2000, 16, dropout=0.2)) 
model.add(LSTM(16, dropout_W=0.2, dropout_U=0.2)) 
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, Y_train, batch_size=8, epochs=3,
          validation_data=(X_test, Y_test))
score, acc = model.evaluate(X_test, Y_test,
                            batch_size=8)
print('Test score:', score)
print('Test accuracy:', acc)


print("Generating test predictions...")
#preds15 = model.predict_classes(X_test, verbose=0)
#acc15 = accuracy_score(test['Label'], preds15)

Build model...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(16, dropout=0.2, recurrent_dropout=0.2)`
  # This is added back by InteractiveShellApp.init_path()


Train...
Train on 39760 samples, validate on 9940 samples
Epoch 1/3
39760/39760 [==============================] - 306s 8ms/step - loss: 0.6910 - acc: 0.5344 - val_loss: 0.6910 - val_acc: 0.5344
Epoch 2/3
39760/39760 [==============================] - 308s 8ms/step - loss: 0.6866 - acc: 0.5483 - val_loss: 0.6933 - val_acc: 0.5222
Epoch 3/3
 9160/39760 [=====>........................] - ETA: 3:44 - loss: 0.6712 - acc: 0.5818